In [9]:
%run "libraries.ipynb"

from IPython.display import display, HTML

In [10]:
pages_editors_graph = nx.read_gexf("data/pages-editors.gexf")

In [11]:
bots = [ n for n in pages_editors_graph.nodes() if ("bot" in n.lower()) and not("p:") in n ]
print len(bots)

565


In [12]:
def table_to_html(data, cols=[]):
    html = "<table>"
    html += "<tr>"
    for column_content in cols:
        html += "<th>%s</th>" % (column_content)
    html += "</tr>"
    
    for d in data:
        html += "<tr>"
        for column_content in d:
            html += "<td>%s</td>" % (column_content)
        html += "</tr>"
    html += "</table>"
        
    return HTML(html)

In [13]:
data = []

bots = sorted(bots, key=lambda b: -pages_editors_graph.node[b]["revisions"])

g = pages_editors_graph

for b in bots:
    link = u"<a href=\"http://en.wikipedia.org/wiki/User:{0}\" target=\"_blank\">{0}</a>".format(b.split(":")[1])
    r  = g.node[b]["revisions"]
    p  = len(pages_editors_graph[b])
    m  = np.mean(map(lambda p: float(g[b][p]["revisions"])/float(g.node[p]["revisions"]), g[b]))
    rp = float(r)/float(p)
    f_rev = g.node[b]["first revision"]
    l_rev = g.node[b]["last revision"]
    data.append([link, r, p, rp, m, f_rev, l_rev])

count_all_rev = sum([ g.node[p]["revisions"] for p in pages_editors_graph.nodes() if g.node[p]["type"] == "page" ])
count_bots_rev = sum([ b[1] for b in data ])

print "total number of revisions: %s" % (count_all_rev)
print "revisions by all bots: %s (%s)" % (count_bots_rev, float(count_bots_rev)/float(count_all_rev) * 100)

display(table_to_html(data, ["bot name", "revisions", "edited pages", "revisions/page", "mean contribution", "first activity", "last activity"]))

total number of revisions: 697011
revisions by all bots: 59107 (8.48006702907)


bot name,revisions,edited pages,revisions/page,mean contribution,first activity,last activity
ClueBot NG,9082,589,15.4193548387,0.0170719416878,2010-11-02 20:01:51,2015-03-31 22:55:36
ClueBot,5688,474,12.0,0.00932075478177,2007-08-04 00:48:06,2010-12-02 12:36:14
SmackBot,5060,882,5.73696145125,0.0172569990188,2006-03-04 21:44:13,2011-07-10 21:41:14
AnomieBOT,2492,615,4.05203252033,0.00771577500964,2008-08-26 20:03:25,2015-04-01 01:31:28
AntiVandalBot,1762,260,6.77692307692,0.00374804563134,2006-06-13 03:00:53,2007-04-02 15:47:20
Yobot,1551,704,2.203125,0.011160425198,2008-07-12 09:10:52,2015-04-01 06:41:21
VoABot II,1245,245,5.08163265306,0.00329798635512,2006-08-07 22:54:27,2009-10-06 13:14:59
Cydebot,1131,530,2.13396226415,0.0128729030396,2006-04-21 17:54:25,2015-03-21 18:02:38
Addbot,1108,914,1.21225382932,0.0108197631683,2008-05-10 15:28:25,2013-08-21 03:36:45
Tawkerbot2,1039,200,5.195,0.00266768603938,2006-03-06 09:29:24,2006-10-25 03:34:41


## references

- https://books.google.fr/books?id=t5RpAgAAQBAJ (p.137-?)
- http://networkcultures.org/blog/publication/critical-point-of-view-a-wikipedia-reader/
- http://en.wikipedia.org/wiki/Wikipedia:Bot_policy
- http://www.davidribes.com/storage/Ribes%20Bowker%20-%20From%20Meaning%20to%20Machine.pdf
- http://www.davidribes.com/storage/Ribes%20Jackson%20et%20al%20-%20Artifacts%20that%20Organize.pdf
- http://www.davidribes.com/storage/Geiger%20Ribes%20-%20Sustaining%20Order%20in%20Wikipedia.pdf